In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math
import os
#import pandas as pd

This function will open the data

In [ ]:
def getData(fname, path):

    dataFile = open(path+'/'+ fname,"r")
    #dataFile = open("/content/drive/Shareddrives/RFID_Capstone/16x16 Experiment Hex/Y0/x0y0.csv","r")
    timestamp = []
    EPC = []
    Antenna = []
    RSSI = []
    PhaseAngle = []

    dataFile.readline()
    dataFile.readline()
    dataFile.readline()
    # for loop, it will run though every line in the file
    for line in dataFile:
        line.encode('utf-8').strip()
        line = line.strip()
        Timestamp, epc, tid, antenna,Rssi, frequence, hostname, PA, DF= line.split(',')

        timestamp.append(Timestamp)
        EPC.append(epc)
        Antenna.append(int(antenna))
        RSSI.append(float(Rssi))
        #PhaseAngle.append(float(PA))

    #closing the file so it doesn't take up unnecessary RAM
    dataFile.close()
    return timestamp, EPC, Antenna, RSSI #, PhaseAngle
    #return the list we got from the file

In [ ]:
def variance(data, mean, ddof=0):
    n = len(data)
    return sum((x - mean) ** 2 for x in data) / (n - ddof)

def stdev(data):
    mean = findSum(data)/len(data)
    var = variance(data, mean)
    std_dev = math.sqrt(var)
    return std_dev, mean

def findSum(dataList):
    sum = 0
    for i in range(len(dataList)):
        sum += dataList[i]
    return int(sum)

In [ ]:
def dataStats(index, timestamp, UID, EPC, Antenna, RSSI, p):
    xBarSignalLisst = []
    XbarPA = []
    numDetection = []
    rssi_list = []
    for x in range(len(index)):
        #initial some varibles
        sumMean = 0
        sumAngle = 0
        j = 0
        # find the index i where timestamp, EPC and anttenna mataches
        for i in range(len(timestamp)):
            if index[x] == timestamp[i] and UID == EPC[i] and  Antenna[i] == p:
                rssi_list.append(RSSI[i])
                #add the signal strength in to sum
                sumMean += RSSI[i]
                j += 1
        # if there is dection mean of the sigal value divided by the number of detection and append that else append 0
        if j > 0:
            Xbar_meanIndex = sumMean / j
            xBarSignalLisst.append(float(Xbar_meanIndex))
        else:
            xBarSignalLisst.append(float(0))
        #numDetection.append(int(j))

    if len(xBarSignalLisst) > 0:
        RSSI_sd, RSSI_mean = stdev(xBarSignalLisst)

    if len(rssi_list) > 0:
        RSSI_min = min(rssi_list)
        RSSI_max = max(rssi_list)

    return RSSI_sd, RSSI_mean

In [ ]:
def data(index, timestamp, UID, EPC, Antenna, RSSI,location, x, y):

    RSSI_sd1, RSSI_mean1 = dataStats(index, timestamp, UID, EPC, Antenna, RSSI, 1)
    RSSI_sd2, RSSI_mean2 = dataStats(index, timestamp, UID, EPC, Antenna, RSSI, 2)
    RSSI_sd3, RSSI_mean3 = dataStats(index, timestamp, UID, EPC, Antenna, RSSI, 3)
    RSSI_sd4, RSSI_mean4 = dataStats(index, timestamp, UID, EPC, Antenna, RSSI, 4)

    #print(RSSI_sd4, RSSI_mean4)

    alldata = [f'{RSSI_sd1},{RSSI_mean1},{RSSI_sd2},{RSSI_mean2},{RSSI_sd3},{RSSI_mean3},{RSSI_sd4},{RSSI_mean4},{x},{y}']

    #print(alldata)
    return alldata


In [ ]:
def getLetter(dir_name):
    #dir_name = input("Enter dir to sort:" )
    if(len(dir_name)) > 2:
        IndexNum = int(dir_name[1:3])
    else:
        IndexNum = int(dir_name[1])
    letter =  str(chr(96+IndexNum).upper())
    return letter, IndexNum

def getNum(fname, y):
    if y >= 10:
        if len(fname) >= 10:
          num = int(fname[1:3])
        else:
          num = int(fname[1:2])
    else:
      if len(fname) >= 9:
          num = int(fname[1:3])
      else:
          num = int(fname[1:2])
    return num

def unique(data):
    unique = []
    for line in data:
        if line not in unique:
            unique.append(line)
    return unique

def newtimestamp(timestamp):
    for i in range(len(timestamp)):
        timestamp[i] = timestamp[i][11:19]
    UTimestamp= unique(timestamp)
    return UTimestamp, timestamp


In [ ]:
def createFile(data, fname):
    print(fname)
    outF = open('/content/drive/Shareddrives/RFID_Capstone/11x11Ouput/round2/'+ str(fname) + '_' +"Condense_data.csv", "w")
    textline = str("RSSI_sd1" + "," + "RSSI_mean1" + "," +
        "RSSI_sd2" + "," + "RSSI_mean2"  + "," +
        "RSSI_sd3" + "," + "RSSI_mean3"  + "," +
        "RSSI_sd4" + "," + "RSSI_mean4"  + "," +
         "X" + ',' + "Y" + '\n')
    outF.write(textline)
    for line in data:
        # write line to output file
        textline = str(line) + '\n'
        outF.write(textline)
    outF.close()
    return

For some reason, when you use os it will create a checkpoint, we do not want this checkpoint to be in the file when we run the main funciton

In [ ]:
import os
import glob
import pandas as pd

def combine_data():
    os.chdir("/content/drive/Shareddrives/RFID_Capstone/11x11Ouput/")
    extension = 'csv'
    all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
    combined_csv.to_csv( "/content/drive/Shareddrives/RFID_Capstone/11x11Ouput/Condense_Data.csv", index=False, encoding='utf-8-sig')
    print(combined_csv)
    return

In [ ]:
def main(dir, name):
    dataSheet = []
    Tag = "E2801170000002150E68ED20"
    letter, y = getLetter(name)
    #print(name)
    dir_list = os.listdir(dir)
    #print(dir_list)
    for i in range(len(dir_list)):
        num = getNum(dir_list[i], y)
        if num < 10:
            location = str(letter) + str(0)+ str(num)
        else:
            location = str(letter) + str(num)
        timestamp, EPC, Antenna, RSSI = getData(dir_list[i], dir)
        # print(Antenna)
        UTimestamp, timestamp = newtimestamp(timestamp)
        dataStats = data(UTimestamp[1:(len(UTimestamp)-1)], timestamp, Tag, EPC, Antenna, RSSI, location, num, y)
        dataSheet += dataStats
        #print(dataSheet)

    createFile(dataSheet, name )
    print("Sorting Completed")
    #print(dataSheet)


In [ ]:
name = ['Y0','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10']#,'Y11','Y12','Y13','Y14','Y15','Y16'
if __name__ == "__main__":
  for index in range(len(name)):
    dir = '/content/drive/Shareddrives/RFID_Capstone/11x11 Factorial/Round2/'+ name[index]
    main(dir, name[index])

  print("Combining all processd data")
  combine_data()
  print("Processd data finished combining")

Y0
Sorting Completed
Y1
Sorting Completed
Y2
Sorting Completed
Y3
Sorting Completed
Y4
Sorting Completed
Y5
Sorting Completed
Y6
Sorting Completed
Y7
Sorting Completed
Y8
Sorting Completed
Y9
Sorting Completed
Y10
Sorting Completed
Combining all processd data
     RSSI_sd1  RSSI_mean1   RSSI_sd2  RSSI_mean2   RSSI_sd3  RSSI_mean3  \
0    0.641192  -56.827586   1.629036  -61.379310   1.451936  -66.965517   
1    0.924029  -65.758621   0.514498  -46.586207  31.500867  -43.344828   
2    0.626412  -64.428571   0.402979  -52.321429   2.202062  -64.357143   
3    0.679335  -48.678571  16.852237  -60.642857   2.384387  -59.750000   
4    0.862500  -62.321429   1.143203  -57.928571  32.348547  -40.178571   
..        ...         ...        ...         ...        ...         ...   
6    1.304281  -64.931034  11.930865  -60.482759   0.994561  -50.517241   
7    1.291237  -62.500000   0.911208  -64.357143   0.463096  -51.357143   
8   16.909821  -61.862069   0.990681  -65.724138   0.822927  -65.

In [ ]:
combine_data()

      RSSI_sd1  RSSI_mean1   RSSI_sd2  RSSI_mean2   RSSI_sd3  RSSI_mean3  \
0     0.641192  -56.827586   1.629036  -61.379310   1.451936  -66.965517   
1     0.924029  -65.758621   0.514498  -46.586207  31.500867  -43.344828   
2     0.626412  -64.428571   0.402979  -52.321429   2.202062  -64.357143   
3     0.679335  -48.678571  16.852237  -60.642857   2.384387  -59.750000   
4     0.862500  -62.321429   1.143203  -57.928571  32.348547  -40.178571   
..         ...         ...        ...         ...        ...         ...   
116   1.304281  -64.931034  11.930865  -60.482759   0.994561  -50.517241   
117   1.291237  -62.500000   0.911208  -64.357143   0.463096  -51.357143   
118  16.909821  -61.862069   0.990681  -65.724138   0.822927  -65.689655   
119   1.894408  -64.321429   0.978639  -65.357143  20.955251  -60.250000   
120   1.076945  -65.517241  12.404745  -64.689655   1.162236  -63.793103   

      RSSI_sd4  RSSI_mean4   X   Y  
0     1.568278  -63.896552   4   0  
1    25.26446